In [211]:
import random
import math
class queen_solve:
    desk_size = 8
    cell_size = desk_size * desk_size
    population = list()
    population_count = 50
    crossingover_p = 0.5
    def row_col_fitness(self, count):
        return math.factorial(count) if count > 0 else 24
    
    def diags_fitness(self, count):
        return math.factorial(count)
    
    def fitness(self, cell):
        res = 0
        for i in range(self.desk_size):
            res = res + self.row_col_fitness(sum(cell[i:self.cell_size:self.desk_size]))
            res = res + self.row_col_fitness(sum(cell[i*self.desk_size:(i + 1)*self.desk_size]))
        for i in range(-self.desk_size, self.desk_size):
            res = res + self.diags_fitness(sum(self.get_main_diag(cell, i)))
            res = res + self.diags_fitness(sum(self.get_sub_diag(cell, i)))
        return abs(res - 48)

    def get_main_diag(self, cell, diag_n):
        '''get diag parallel to main'''
        res = list()
        if diag_n < 0:
            for j in range(abs(diag_n), self.desk_size):
                i = j + diag_n
                res.append(cell[i * self.desk_size + j])
        else:
            for j in range(0, self.desk_size - diag_n):
                i = j + diag_n
                res.append(cell[i * self.desk_size + j])
        return res
    
    def get_sub_diag(self, cell, diag_n):
        '''get diag parallel to sub'''
        res = list()
        if diag_n < 0:
            for j in range(0, self.desk_size + diag_n):
                i = self.desk_size + diag_n - j - 1
                res.append(cell[i * self.desk_size + j])
        else:
            for j in range(diag_n, self.desk_size):
                i = self.desk_size + diag_n - j - 1
                res.append(cell[i * self.desk_size + j])
        return res
        
    def to_str(self, cell):
        res = ""
        for i in range(self.desk_size):
            for j in range(self.desk_size):
                res = res + ("Q " if cell[i * self.desk_size + j] == 1 else "+ ")
            res = res + '\n'
        return res
    
    def generate(self, p):
        cell = list()
        for i in range(self.cell_size):
            cell.append(1 if random.uniform(0, 1) <= p else 0)
        return cell
        
    def get_population_probs(self, population):
        probs = list()
        for cell in population:
            probs.append(self.fitness(cell))
        # normalize:
        sum_p = sum(probs)
        last_p = 0
        for i in range(len(probs)):
            probs[i] = last_p + probs[i] / sum_p
            last_p = probs[i]
        return probs
        
    def ring_select(self, probs, p):
        for i in range(len(probs)):
            if p < probs[i]:
                return i
        return len(probs) - 1
        
    def one_point_crossing(self, a, b):
        k = random.randrange(len(a)) 
        return a[0:k] + b[k: len(a)]
    
    def crossing_step(self):
        new_population = list()
        probs = self.get_population_probs(population)
        for i in range(population_count):
            a = ring_select(probs, random.random())
            b = ring_select(probs, random.random())
            while a == b:
                b = ring_select(probs, random.random())
            new_population.append(self.one_point_crossing(self.population[a], self.population[b]))
    
    def reduction(self, population):
        fitness = list()
        for cell in population:
            fitness.append(self.fitness(cell))
        pairs = zip(fitness, population)
        pairs.sort(key=lambda x: x[0], reverse=True)
        return pairs[0: self.population_count]
        
            
        

In [204]:
# 
solver = queen_solve()
cell = solver.generate(0)
print(solver.to_str(cell))
print(solver.fitness(cell))

+ + + + + + + + 
+ + + + + + + + 
+ + + + + + + + 
+ + + + + + + + 
+ + + + + + + + 
+ + + + + + + + 
+ + + + + + + + 
+ + + + + + + + 

368


In [206]:
# check ring select
solver = queen_solve()
population = list()
for i in range(10):
    population.append(solver.generate(0.1))
    
probs = solver.get_population_probs(population)
print(probs, solver.ring_select(probs, 1))


[0.1658682634730539, 0.2532934131736527, 0.33353293413173657, 0.4059880239520959, 0.5047904191616768, 0.6568862275449103, 0.6766467065868265, 0.787425149700599, 0.8862275449101797, 1.0] 9
